In [1]:
from ai import ai_by_score, dprint
from marubatsu import Marubatsu
from copy import deepcopy

@ai_by_score
def ai_scout(mb, debug=False, shortest_victory=False,
             init_ab=False, use_tt=False, ai_for_mo=None,
             params={}, sort_allnodes=False, calc_count=False):        
    count = 0
    def ab_search(mborig, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        count += 1
        if mborig.status == Marubatsu.CIRCLE:
            return (11 - mborig.move_count) / 2 if shortest_victory else 1
        elif mborig.status == Marubatsu.CROSS:
            return (mborig.move_count - 10) / 2 if shortest_victory else -1
        elif mborig.status == Marubatsu.DRAW:
            return 0
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if ai_for_mo is not None:
            if sort_allnodes:
                score_by_move = ai_for_mo(mborig, analyze=True, **params)["score_by_move"]
                score_by_move_list = sorted(score_by_move.items(), key=lambda x:x[1], reverse=True)
                legal_moves = [x[0] for x in score_by_move_list]
            else:
                legal_moves = mborig.calc_legal_moves()
                bestmove = ai_for_mo(mborig, rand=False, **params)
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]

        if mborig.turn == Marubatsu.CIRCLE:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, tt, alpha, beta)
            alpha = max(alpha, score)
            if score < beta:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    score = max(score, ab_search(mb, tt, alpha, alpha + 1))
                    if score >= beta:
                        break
                    elif score > alpha:
                        score = max(score, ab_search(mb, tt, alpha, beta))
                        if score >= beta:
                            break
                        alpha = max(alpha, score)
        else:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, tt, alpha, beta)
            beta = min(beta, score)
            if score > alpha:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    score = min(score, ab_search(mb, tt, beta - 1, beta))
                    if score <= alpha:
                        break
                    elif score < beta:
                        score = min(score, ab_search(mb, tt, alpha, beta))
                        if score <= alpha:
                            break
                        beta = min(beta, score)   
            
        if use_tt:
            from util import calc_same_boardtexts

            boardtxtlist = calc_same_boardtexts(mborig)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt in boardtxtlist:
                tt[boardtxt] = (lower_bound, upper_bound)
        return score
                
    min_score = -2 if shortest_victory else -1
    max_score = 3 if shortest_victory else 1
    alpha = min_score if init_ab else float("-inf")
    beta = max_score if init_ab else float("inf")

    score = ab_search(mb, {}, alpha=alpha, beta=beta)
    dprint(debug, "count =", count)
    if calc_count:
        return count
    if mb.turn == Marubatsu.CIRCLE:
        score *= -1
    return score

In [2]:
from util import Check_solved
from ai import ai2s, ai14s

for ai_for_mo in [None, ai2s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                for sort_allnodes in [False, True]:
                    print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, use_tt: {use_tt}, init_ab: {init_ab} sort: {sort_allnodes}")
                    params = {
                        "ai_for_mo": ai_for_mo,
                        "shortest_victory": shortest_victory,
                        "init_ab": init_ab,
                        "use_tt": use_tt,
                        "sort_allnodes": sort_allnodes,
                    }
                    Check_solved.is_strongly_solved(ai_scout, params=params)

ai_for_mo: None, sv: False, use_tt: False, init_ab: False sort: False


100%|██████████| 431/431 [00:04<00:00, 104.11it/s]


431/431 100.00%
ai_for_mo: None, sv: False, use_tt: False, init_ab: False sort: True


100%|██████████| 431/431 [00:04<00:00, 99.15it/s] 


431/431 100.00%
ai_for_mo: None, sv: False, use_tt: False, init_ab: True sort: False


...

431/431 100.00%
ai_for_mo: <function ai14s at 0x0000016BFAD0A3E0>, sv: True, use_tt: True, init_ab: True sort: False


100%|██████████| 431/431 [00:04<00:00, 102.03it/s]


431/431 100.00%
ai_for_mo: <function ai14s at 0x0000016BFAD0A3E0>, sv: True, use_tt: True, init_ab: True sort: True


100%|██████████| 431/431 [00:04<00:00, 104.05it/s]

431/431 100.00%


In [3]:
from util import load_mblist
from ai import ai10s, ai_abs_all

mblist = load_mblist("../data/mblist_by_board_min.dat")
for ai_for_mo in [ai10s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, init_ab: {init_ab}, use_tt: {use_tt}")
                count = 0
                count2 = 0
                for mb in mblist:
                    count += ai_abs_all(mb, ai_for_mo=ai_for_mo,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        sort_allnodes=False, calc_score=True, calc_count=True)
                    count2 += ai_scout(mb, ai_for_mo=ai_for_mo,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        sort_allnodes=False, calc_score=True, calc_count=True)
                print("abs count=", count)
                print("scout count=", count2)

ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: False, init_ab: False, use_tt: False
abs count= 13866
scout count= 14726
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: False, init_ab: True, use_tt: False
abs count= 8769
scout count= 9400
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: False, init_ab: False, use_tt: True
abs count= 10384
scout count= 10392
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: False, init_ab: True, use_tt: True
abs count= 6651
scout count= 6837
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: True, init_ab: False, use_tt: False
abs count= 15135
scout count= 15645
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: True, init_ab: True, use_tt: False
abs count= 13145
scout count= 14052
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: True, init_ab: False, use_tt: True
abs count= 11091
scout count= 11135
ai_for_mo: <function ai10s at 0x0000016BFAD09EE0>, sv: True, init_ab: True, use_tt: True
abs count= 9743
scout count=

In [4]:
from util import load_bestmoves
from ai import ai_gt6

bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board.dat")
params = {"bestmoves_by_board": bestmoves_by_board}
for ai_for_mo in [ai_gt6]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, init_ab: {init_ab}, use_tt: {use_tt}")
                count = 0
                count2 = 0
                for mb in mblist:
                    count += ai_abs_all(mb, ai_for_mo=ai_for_mo, params=params,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        calc_score=True, calc_count=True)
                    count2 += ai_scout(mb, ai_for_mo=ai_for_mo, params=params,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        calc_score=True, calc_count=True)
                print("abs count=", count)
                print("scout count=", count2)

ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: False, init_ab: False, use_tt: False
abs count= 13151
scout count= 13151
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: False, init_ab: True, use_tt: False
abs count= 8225
scout count= 8225
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: False, init_ab: False, use_tt: True
abs count= 10035
scout count= 10035
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: False, init_ab: True, use_tt: True
abs count= 6080
scout count= 6080
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: False, use_tt: False
abs count= 14404
scout count= 14389
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: True, use_tt: False
abs count= 13215
scout count= 13345
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: False, use_tt: True
abs count= 10634
scout count= 10590
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: True, use_tt: True
abs count= 9691
scou

In [5]:
bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board_shortest_victory.dat")
params = {"bestmoves_by_board": bestmoves_by_board}
for ai_for_mo in [ai_gt6]:
    for shortest_victory in [True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, init_ab: {init_ab}, use_tt: {use_tt}")
                count = 0
                count2 = 0
                for mb in mblist:
                    count += ai_abs_all(mb, ai_for_mo=ai_for_mo, params=params,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        calc_score=True, calc_count=True)
                    count2 += ai_scout(mb, ai_for_mo=ai_for_mo, params=params,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        calc_score=True, calc_count=True)
                print("abs count=", count)
                print("scout count=", count2)

ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: False, use_tt: False
abs count= 11106
scout count= 11106
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: True, use_tt: False
abs count= 9718
scout count= 9718
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: False, use_tt: True
abs count= 8955
scout count= 8955
ai_for_mo: <function ai_gt6 at 0x0000016BFAD0AB60>, sv: True, init_ab: True, use_tt: True
abs count= 7706
scout count= 7706
